# Segmenting and Clustering Neighborhoods in Toronto

##  Author: Ziqi (Jazmine) Liu 

### This lab will explore and cluster the neighborhoods in Toronto.

#### To create the above dataframe: 
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.<br>
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.<br>
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.<br>
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.<br>
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.<br>
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.  

#### Import All Needed Libraries 

In [1]:
import pandas as pd
import numpy as np
import requests 
import matplotlib as plt

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!conda install -c anaconda beautifulsoup4 --yes

!conda install -c http://conda.anaconda.org/r r-xml --yes

from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('All libraries have been imported')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi               anaconda::certifi-2020.4.5.1-py37_0 --> conda-forge::certifi-2020.4.5.1-py37hc8dfbb8_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n ba

## Part 1
### Build the code to scrape the Wikipedia page

 https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. 

In [2]:
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source= requests.get(link).text

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(source, 'html.parser')

In [4]:
table = soup.find('table')

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [5]:
#define the dataframe columns
column_names = ['Postalcode','Borough','Neighborhood']
Toronto_df = pd.DataFrame(columns = column_names)

In [6]:
res = []
for tr in table.tbody.find_all("tr"):
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    if row != [] and row[1] != "Not assigned":
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

In [7]:
Toronto_df= pd.DataFrame(res, columns = column_names)
Toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [8]:
#Clean the table by removing all "\n"
Toronto_df["Postalcode"] = Toronto_df["Postalcode"].str.replace('\n',"")
Toronto_df["Borough"] = Toronto_df["Borough"].str.replace('\n',"")
Toronto_df["Neighborhood"] = Toronto_df["Neighborhood"].str.replace('\n',"")

Toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [9]:
#drop not assigned indexes
Toronto_df.drop(Toronto_df[Toronto_df['Borough'] == 'Not assigned'].index, inplace = True)
Toronto_df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
#reset index
Toronto_df.reset_index(drop=True, inplace= True)
Toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Use the .shape method to print the number of rows of my dataframe.

In [11]:
print("Data dimensions:",Toronto_df.shape)

Data dimensions: (103, 3)


## Part 2
### Built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name.
- In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [12]:
Geo_data= "https://cocl.us/Geospatial_data"

In [13]:
df2 = pd.read_csv(Geo_data)

In [14]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
#check the previous dataframe 
Toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Combine two dataframes "df2" and "Toronto_df"

In [16]:
Combined_df = pd.concat([Toronto_df, df2], axis=1, sort=False)
Combined_df.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476


#### Remove the duplicate column "Postal Code"

In [17]:
Combined_df.drop("Postal Code", axis=1, inplace=True)
Combined_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


## Part3

### Explore and cluster the neighborhoods in Toronto.

#### Obtain the latitude and longitude values of Toronto:
- In order to define an instance of the geocoder, we need to define a user_agent. 
- I will name my agent trt_explorer, as shown below:

In [18]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


 #### 3.1 Create a map of Toronto with neighborhoods superimposed on top.

In [19]:
# create map of Toronto using latitude and longitude values
map_trt = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(
    Combined_df['Latitude'],
    Combined_df['Longitude'], 
    Combined_df['Borough'], 
    Combined_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trt)  
    
map_trt

#### For illustration purposes, I will simplify the above map and segment and cluster only the neighborhoods in Toronto:
- I will achieve it by slicing the original dataframe and create a new dataframe of the Toroto data.

In [20]:
Toronto_data = Combined_df[Combined_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975


- Obtain the geographical coordinates of Toronto

In [21]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


- Generate the new map for Toronto

In [22]:
# create map of Toronto using latitude and longitude values
map_trt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'],Toronto_data['Longitude'],Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='deeppink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trt)  
    
map_trt

#### Next, I will start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [23]:
CLIENT_ID = 'R2EX14GSFQBP04WWWQE2APM4A0YOECWDET4X4GROT03SBICE'
CLIENT_SECRET = 'V3MUSKLZLBBWFBH0F3RYTGF4UU30ZGEAXIFEJN2LMPIIUOOZ'
VERSION = '20200510'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R2EX14GSFQBP04WWWQE2APM4A0YOECWDET4X4GROT03SBICE
CLIENT_SECRET:V3MUSKLZLBBWFBH0F3RYTGF4UU30ZGEAXIFEJN2LMPIIUOOZ


#### Explore the first neighbourhood in my dataframe
- Obtain the neighbourhood's name:

In [24]:
Toronto_data.loc[0,'Neighborhood']

'Regent Park, Harbourfront'

- Obtain the neighbouthood's latitude and longitude values:

In [25]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = Toronto_data.loc[0, 'Longitude']

neighborhood_name = Toronto_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


- Obtain the top 100 venues that are in Toronto within a radius of 500 meters:

- Create the GET request URL. Name your URL **url**.

In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL 

'https://api.foursquare.com/v2/venues/explore?&client_id=R2EX14GSFQBP04WWWQE2APM4A0YOECWDET4X4GROT03SBICE&client_secret=V3MUSKLZLBBWFBH0F3RYTGF4UU30ZGEAXIFEJN2LMPIIUOOZ&v=20200510&ll=43.7635726,-79.1887115&radius=500&limit=100'

- Send the GET request and examine the resutls:

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb8a3fe77af03002981561a'},
 'response': {'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.76678992471017,
          'lng': -79.1911511887

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

- clean the json and structure it into a *pandas* dataframe.

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
3,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406
4,Woburn Medical Centre,Medical Center,43.766631,-79.192286


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


#### 3.2 Explore Neighborhoods in Toronto

- Create a function to repeat the same process to all the neighborhoods in Toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
             # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

- Create a new dataframe called *Toronto_venues*

In [32]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

- Check the size of the resulting data frame:

In [33]:
print('Data dimension:',Toronto_venues.shape)
Toronto_venues.head()

Data dimension: (740, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,"Regent Park, Harbourfront",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


- Check how many venues were returned for each neighborhood:

In [34]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
"Brockton, Parkdale Village, Exhibition Place",40,40,40,40,40,40
Business reply mail Processing Centre,4,4,4,4,4,4
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,6,6,6,6,6,6
Christie,3,3,3,3,3,3
Church and Wellesley,7,7,7,7,7,7
"Commerce Court, Victoria Hotel",2,2,2,2,2,2
Davisville,4,4,4,4,4,4


- Find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 198 uniques categories.


#### 3.3 Analyze Each Neighborhood

In [36]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
Toronto_onehot.shape

(740, 198)

- Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.025000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,...,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.025,0.0,0.0
2,Business reply mail Processing Centre,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
4,Central Bay Street,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0


In [39]:
#confirm the new size
Toronto_grouped.shape

(39, 198)

- Print each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                       venue  freq
0                       Park   1.0
1  Latin American Restaurant   0.0
2             Lingerie Store   0.0
3               Liquor Store   0.0
4                     Lounge   0.0


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.08
2      Brewery  0.05
3       Bakery  0.05
4    Gastropub  0.05


----Business reply mail Processing Centre----
               venue  freq
0        Pizza Place  0.25
1     Sandwich Place  0.25
2  Mobile Phone Shop  0.25
3           Bus Line  0.25
4        Yoga Studio  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Light Rail Station  0.11
1                 Spa  0.06
2          Comic Shop  0.06
3                Park  0.06
4         Pizza Place  0.06


----Central Bay Street----
           venue  freq
0       Pharmacy  0.17
1  Grocery Store  0

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

- Create the new dataframe and display the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Park,Women's Store,Cuban Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Middle Eastern Restaurant,Bookstore,Pet Store
2,Business reply mail Processing Centre,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Yoga Studio,Spa,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Burrito Place
4,Central Bay Street,Butcher,Grocery Store,Bank,Pizza Place,Pharmacy,Coffee Shop,Concert Hall,Deli / Bodega,Eastern European Restaurant,Doner Restaurant


### 3.4 Cluster Neighborhoods

- Run *k*-means to cluster the neighborhood into 5 clusters.

In [48]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() 

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,4,Mexican Restaurant,Electronics Store,Intersection,Breakfast Spot,Bank,Rental Car Location,Medical Center,Doner Restaurant,Dog Run,Discount Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,4,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Gas Station,Diner,Curling Ice
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,4,General Entertainment,College Stadium,Skating Rink,Café,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,4,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Breakfast Spot,Burger Joint,Camera Store,Pizza Place,Pharmacy,Bank,Grocery Store
4,M4E,East Toronto,The Beaches,43.786947,-79.385975,4,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run


#### Visualize the Result:

In [49]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'],Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3.5 Examine Clusters

##### Cluster 1

In [50]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Curling Ice,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
8,Downtown Toronto,0,Airport,Park,Snack Place,Women's Store,Cuban Restaurant,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
26,Central Toronto,0,Park,Basketball Court,Construction & Landscaping,Bakery,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run


##### Cluster 2

In [51]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,1,Park,Women's Store,Cuban Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
24,Central Toronto,1,Park,Women's Store,Pool,Cuban Restaurant,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


##### Cluster 3

In [52]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,2,Construction & Landscaping,Baseball Field,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store


##### Cluster 4

In [53]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Downtown Toronto,3,Baseball Field,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner


##### Cluster 5

In [54]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Mexican Restaurant,Electronics Store,Intersection,Breakfast Spot,Bank,Rental Car Location,Medical Center,Doner Restaurant,Dog Run,Discount Store
1,Downtown Toronto,4,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Gas Station,Diner,Curling Ice
2,Downtown Toronto,4,General Entertainment,College Stadium,Skating Rink,Café,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store
3,Downtown Toronto,4,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Breakfast Spot,Burger Joint,Camera Store,Pizza Place,Pharmacy,Bank,Grocery Store
4,East Toronto,4,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
6,Downtown Toronto,4,Butcher,Grocery Store,Bank,Pizza Place,Pharmacy,Coffee Shop,Concert Hall,Deli / Bodega,Eastern European Restaurant,Doner Restaurant
9,West Toronto,4,Park,Grocery Store,Bank,Hotel,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
10,Downtown Toronto,4,Skating Rink,Bus Stop,Pharmacy,Diner,Curling Ice,Athletics & Sports,Beer Store,Park,Concert Hall,Department Store
11,West Toronto,4,Health Food Store,Trail,Pub,Cuban Restaurant,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
12,East Toronto,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Juice Bar,Restaurant,Brewery,Pub
